In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import joblib

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/twtr_dataset.csv')

In [ ]:
data = df.dropna(how='any')

In [ ]:
def preprocess_text(df):
    # Define stopwords and clean text
    add_stopwords = ["http", "https", "amp", "rt", "t", "c", "the"]
    df['clean_text'] = df['clean_text'].str.replace(r'\W', ' ')  # Removing non-word characters
    df['clean_text'] = df['clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in add_stopwords]))
    return df

In [ ]:
data = preprocess_text(data)

# Encode labels
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['category'])

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(data['clean_text'], data['label'], test_size=0.3, random_state=100)

# Vectorization and model training
vectorizer = CountVectorizer(max_features=30000, min_df=5)
lr = LogisticRegression(max_iter=20)

pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', lr)
])

# Grid search for hyperparameter tuning
param_grid = {
    'classifier__C': [0.1, 0.3, 0.5],
    'classifier__penalty': ['l2'],
    'classifier__solver': ['liblinear']
}

grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

<ipython-input-4-2a09630f8d85>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text'] = df['clean_text'].str.replace(r'\W', ' ')  # Removing non-word characters
<ipython-input-4-2a09630f8d85>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text'] = df['clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in add_stopwords]))
<ipython-input-5-0f09661e613c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vectorizer',
                                        CountVectorizer(max_features=30000,
                                                        min_df=5)),
                                       ('classifier',
                                        LogisticRegression(max_iter=20))]),
             param_grid={'classifier__C': [0.1, 0.3, 0.5],
                         'classifier__penalty': ['l2'],
                         'classifier__solver': ['liblinear']},
             scoring='accuracy')

In [ ]:
joblib.dump(grid.best_estimator_, 'logistic_regression_model.pkl')

['logistic_regression_model.pkl']

In [ ]:

model = joblib.load('logistic_regression_model.pkl')

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy = {accuracy}")
